In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Resizing, GlobalAveragePooling2D, Concatenate, Input

from keras.models import Model

In [ ]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape((60000, 28,28,1))

x_test = x_test.reshape((10000, 28, 28,1))

In [ ]:
def googlenet():
    def inception_block(x,sizes):
        f1= Conv2D(sizes[0],1,activation='relu')(x)
        f2= Conv2D(sizes[1],1,activation='relu')(x)
        f2= Conv2D(sizes[2],3,padding='same', activation='relu')(f2)

        f3= Conv2D(sizes[3],1,activation='relu')(x)
        f3= Conv2D(sizes[4],5,padding='same', activation='relu')(f3)

        f4=  MaxPooling2D(3, strides=1,padding='same')(x)
        f4= Conv2D(sizes[5],1, activation='relu')(f4)

        output= Concatenate()([f1,f2,f3,f4])
        return output

    input=Input(shape=(28,28,1))
    x= Conv2D(64,7,strides=2, padding='same', activation='relu')(input)
    x= MaxPooling2D(3,strides=2,padding='same')(x)
    x=Conv2D(64,1,activation='relu')(x)
    x= Conv2D(192,3, padding='same',activation='relu')(x)
    x= MaxPooling2D(3, strides=2, padding='same')(x)
    x=inception_block(x,[64,96,128,16,32,32])
    x=inception_block(x,[384,192,384,48,128,128])
    x=GlobalAveragePooling2D()(x)
    x=Dropout(0.4)(x)
    output=Dense(10,activation='softmax')(x)
    model=Model(input,output)
    return model
from tensorflow.keras.utils import to_categorical 
#one-hot encoding of training and testing labels

y_train = to_categorical(y_train)

y_test = to_categorical(y_test)

In [ ]:
googlenet_model = googlenet()

In [ ]:
googlenet_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_105 (Conv2D)            (None, 14, 14, 64)   3200        ['input_8[0][0]']                
                                                                                                  
 max_pooling2d_28 (MaxPooling2D  (None, 7, 7, 64)    0           ['conv2d_105[0][0]']             
 )                                                                                                
                                                                                                  
 conv2d_106 (Conv2D)            (None, 7, 7, 64)     4160        ['max_pooling2d_28[0][0]'] 

In [ ]:
googlenet_model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [ ]:
googlenet_model.fit(x_train, y_train, batch_size=64, epochs=1)

938/938 [==============================] - 354s 376ms/step - loss: 0.2725 - accuracy: 0.9221
